# تحليل المشاعر باستخدام Mixtral-8x7B

هذا المشروع يستخدم نموذج Mixtral-8x7B مع تقنية LoRA لتحليل المشاعر في النصوص العربية والإنجليزية.

## متطلبات RunPod:
- GPU: RTX 4090 أو A100 (24GB+ VRAM)
- RAM: 32GB+
- Storage: 50GB+

In [ ]:
# إعدادات RunPod الأولية
import os
import subprocess
import sys

# التحقق من البيئة
print('=== معلومات البيئة ===')
print(f'Python: {sys.version}')
print(f'Working Directory: {os.getcwd()}')

# إعداد متغيرات البيئة لـ RunPod
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['TRANSFORMERS_CACHE'] = '/workspace/cache'
os.environ['HF_HOME'] = '/workspace/cache'

# إنشاء مجلدات العمل
os.makedirs('/workspace/cache', exist_ok=True)
os.makedirs('/workspace/models', exist_ok=True)
os.makedirs('/workspace/data', exist_ok=True)

print('تم إعداد بيئة RunPod بنجاح!')

In [ ]:
# تحميل Google Drive في Colab (اختياري)
try:
    from google.colab import drive
    drive.mount('/content/drive')
    COLAB_ENV = True
    print('تم تحميل Google Drive بنجاح!')
except ImportError:
    COLAB_ENV = False
    print('البيئة المحلية أو RunPod - لا حاجة لتحميل Drive')

In [ ]:
# تثبيت المكتبات المطلوبة لـ Mixtral
!pip install transformers datasets torch scikit-learn seqeval accelerate bitsandbytes
!pip install --upgrade transformers
!pip install peft
!pip install flash-attn --no-build-isolation

In [ ]:
# استيراد المكتبات
import json
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, TaskType
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import warnings
warnings.filterwarnings('ignore')

print('تم استيراد جميع المكتبات بنجاح!')

In [ ]:
# إعداد المسارات
if 'COLAB_ENV' in locals() and COLAB_ENV:
    # مسارات Google Colab
    BASE_PATH = '/content/drive/MyDrive/sentiment_analysis'
    DATA_PATH = os.path.join(BASE_PATH, 'data')
    MODEL_SAVE_PATH = os.path.join(BASE_PATH, 'models')
else:
    # مسارات RunPod أو البيئة المحلية
    BASE_PATH = '/workspace'
    DATA_PATH = os.path.join(BASE_PATH, 'data')
    MODEL_SAVE_PATH = os.path.join(BASE_PATH, 'models')

# إنشاء المجلدات
os.makedirs(DATA_PATH, exist_ok=True)
os.makedirs(MODEL_SAVE_PATH, exist_ok=True)

print(f'مسار البيانات: {DATA_PATH}')
print(f'مسار حفظ النماذج: {MODEL_SAVE_PATH}')

In [ ]:
def load_jsonl_data(file_path):
    """تحميل بيانات JSONL"""
    data = []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                data.append(json.loads(line.strip()))
        print(f'تم تحميل {len(data)} عينة من {file_path}')
        return data
    except FileNotFoundError:
        print(f'الملف غير موجود: {file_path}')
        return []

def simple_sentiment_analysis(text):
    """تحليل مشاعر بسيط باستخدام الكلمات المفتاحية"""
    positive_words = ['جيد', 'ممتاز', 'رائع', 'أحب', 'جميل', 'good', 'great', 'excellent', 'love', 'amazing']
    negative_words = ['سيء', 'فظيع', 'أكره', 'مروع', 'bad', 'terrible', 'hate', 'awful', 'horrible']
    
    text_lower = text.lower()
    positive_count = sum(1 for word in positive_words if word in text_lower)
    negative_count = sum(1 for word in negative_words if word in text_lower)
    
    if positive_count > negative_count:
        return 2  # إيجابي
    elif negative_count > positive_count:
        return 0  # سلبي
    else:
        return 1  # محايد

def prepare_data_for_model(texts, labels=None):
    """إعداد البيانات للنموذج"""
    if labels is None:
        # إنشاء تسميات تلقائية باستخدام التحليل البسيط
        labels = [simple_sentiment_analysis(text) for text in texts]
    
    return list(zip(texts, labels))

print('تم تعريف دوال تحميل البيانات!')

In [ ]:
# إنشاء بيانات تجريبية للتدريب
sample_texts = [
    'هذا المنتج رائع جداً وأنصح به بشدة',
    'المنتج سيء جداً ولا أنصح بشرائه أبداً',
    'المنتج عادي، لا بأس به',
    'أحب هذا المنتج كثيراً',
    'أكره هذا المنتج',
    'المنتج جيد نوعاً ما',
    'This product is amazing and I highly recommend it',
    'This product is terrible and I hate it',
    'The product is okay, nothing special',
    'I love this product so much',
    'I really hate this product',
    'The product is good but could be better'
] * 100  # تكرار البيانات للحصول على عينة أكبر

# إعداد البيانات
prepared_data = prepare_data_for_model(sample_texts)
texts, labels = zip(*prepared_data)

print(f'عدد العينات: {len(texts)}')
print(f'توزيع التسميات: {np.bincount(labels)}')

In [ ]:
# إعداد tokenizer لـ Mixtral
model_name = 'mistralai/Mixtral-8x7B-Instruct-v0.1'

print('تحميل tokenizer...')
tokenizer = AutoTokenizer.from_pretrained(model_name)

# إضافة pad token إذا لم يكن موجوداً
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f'تم تحميل tokenizer: {model_name}')
print(f'حجم المفردات: {len(tokenizer)}')

In [ ]:
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

print('تم تعريف فئة Dataset!')

In [ ]:
# تقسيم البيانات
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

# إنشاء datasets
train_dataset = SentimentDataset(train_texts, train_labels, tokenizer)
val_dataset = SentimentDataset(val_texts, val_labels, tokenizer)

print(f'عدد عينات التدريب: {len(train_dataset)}')
print(f'عدد عينات التقييم: {len(val_dataset)}')

In [ ]:
# إعداد نموذج Mixtral مع quantization
print('تحميل نموذج Mixtral...')

# إعدادات quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

# تحميل النموذج
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,
    quantization_config=bnb_config,
    device_map='auto',
    torch_dtype=torch.bfloat16
)

# إعداد LoRA
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=['q_proj', 'v_proj', 'k_proj', 'o_proj']
)

# تطبيق LoRA على النموذج
model = get_peft_model(model, lora_config)

print('تم تحميل النموذج وإعداد LoRA بنجاح!')
print(f'عدد المعاملات القابلة للتدريب: {model.num_parameters():,}')

In [ ]:
# إعدادات التدريب المحسنة لـ Mixtral
training_args = TrainingArguments(
    output_dir=os.path.join(MODEL_SAVE_PATH, 'mixtral_sentiment_model'),
    num_train_epochs=2,  # تقليل عدد العصور للنموذج الكبير
    per_device_train_batch_size=1,  # حجم دفعة صغير للذاكرة
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,  # تجميع التدرجات
    warmup_steps=100,
    weight_decay=0.01,
    learning_rate=2e-4,  # معدل تعلم مناسب لـ LoRA
    eval_strategy='steps',
    eval_steps=500,
    logging_strategy='steps',
    logging_steps=100,
    save_strategy='steps',
    save_steps=500,
    logging_dir=os.path.join(MODEL_SAVE_PATH, 'logs'),
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    save_total_limit=2,
    report_to='none',
    dataloader_pin_memory=False,
    fp16=False,  # استخدام bfloat16 بدلاً من fp16
    bf16=True,
    gradient_checkpointing=True,  # توفير الذاكرة
    dataloader_num_workers=0,  # تجنب مشاكل multiprocessing
    remove_unused_columns=False,
    optim='adamw_torch',
    max_grad_norm=1.0,
)

print(f'مجلد حفظ النموذج: {training_args.output_dir}')

In [ ]:
def compute_metrics(eval_pred):
    """حساب مقاييس الأداء"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_score(labels, predictions, average='macro')
    }

In [ ]:
# إعدادات RunPod وتحسينات الذاكرة
import gc
import os

# تنظيف الذاكرة
gc.collect()
torch.cuda.empty_cache()

# إعدادات البيئة لـ RunPod
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# طباعة معلومات GPU
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name()}')
    print(f'CUDA Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB')
    print(f'Available Memory: {torch.cuda.memory_reserved(0) / 1024**3:.1f} GB')
else:
    print('تحذير: لا يوجد GPU متاح!')

In [ ]:
# إنشاء المدرب وبدء التدريب
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# تحسين استخدام الذاكرة
model.gradient_checkpointing_enable()

print('بدء تدريب نموذج Mixtral-8x7B...')
print(f'عدد المعاملات القابلة للتدريب: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}')

# بدء التدريب مع معالجة الأخطاء
try:
    trainer.train()
    print('تم الانتهاء من التدريب بنجاح!')
except Exception as e:
    print(f'خطأ أثناء التدريب: {e}')
    # تنظيف الذاكرة في حالة الخطأ
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
# حفظ النموذج المدرب
print('حفظ النموذج...')

# حفظ النموذج الأساسي
trainer.save_model()

# حفظ LoRA adapters
model.save_pretrained(os.path.join(MODEL_SAVE_PATH, 'mixtral_lora_adapters'))

# حفظ tokenizer
tokenizer.save_pretrained(os.path.join(MODEL_SAVE_PATH, 'mixtral_tokenizer'))

print(f'تم حفظ النموذج في: {MODEL_SAVE_PATH}')
print('يمكنك الآن استخدام النموذج للتنبؤ!')

In [ ]:
def classify_sentiment(text):
    """تصنيف المشاعر للنص المدخل"""
    inputs = tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        padding='max_length',
        max_length=128
    )
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=1).item()
        probabilities = torch.softmax(logits, dim=1)[0]
    
    label_map = {0: 'سلبي', 1: 'محايد', 2: 'إيجابي'}
    confidence = probabilities[prediction].item()
    
    return {
        'label': label_map[prediction],
        'confidence': confidence,
        'probabilities': {
            'سلبي': probabilities[0].item(),
            'محايد': probabilities[1].item(),
            'إيجابي': probabilities[2].item()
        }
    }

In [ ]:
# اختبار النموذج
test_texts = [
    'هذا المنتج رائع جداً وأنصح به',
    'المنتج سيء ولا أنصح بشرائه',
    'المنتج عادي، لا بأس به',
    'This product is amazing!',
    'I hate this product'
]

print('=== اختبار النموذج ===')
for text in test_texts:
    result = classify_sentiment(text)
    print(f'النص: {text}')
    print(f'التصنيف: {result["label"]} (ثقة: {result["confidence"]:.2f})')
    print('---')

### تحميل النموذج المدرب في جلسة جديدة

In [ ]:
# تحميل النموذج المدرب في جلسة جديدة
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel
import torch

# مسارات النموذج
base_model_path = 'mistralai/Mixtral-8x7B-Instruct-v0.1'
lora_path = '/workspace/models/mixtral_lora_adapters'  # أو المسار المحفوظ
tokenizer_path = '/workspace/models/mixtral_tokenizer'

# تحميل tokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

# تحميل النموذج الأساسي
base_model = AutoModelForSequenceClassification.from_pretrained(
    base_model_path,
    num_labels=3,
    torch_dtype=torch.bfloat16,
    device_map='auto'
)

# تحميل LoRA adapters
model = PeftModel.from_pretrained(base_model, lora_path)

print('تم تحميل النموذج المدرب بنجاح!')
print('يمكنك الآن استخدام دالة classify_sentiment للتنبؤ')